[View in Colaboratory](https://colab.research.google.com/github/kimitakem/colab_ex/blob/master/sample.ipynb)

# Tensorflow: Eager executionモード
## 初期化：
- ライブラリのインポート
- Eagerモードのアクティベーションを行う


In [1]:
import tensorflow as tf
tf.enable_eager_execution()

## 環境の確認
- Tensorflowのバージョン
- Eagerモードになっているかの確認

In [2]:
print("Tensorflow Version: {}".format(tf.__version__))
print("Check Eager Mode: {}".format(tf.executing_eagerly()))
tfe = tf.contrib.eager


Tensorflow Version: 1.9.0
Check Eager Mode: True


## GradientTapeを使った微分演算
- 変数の定義：`tf.contrib.eager.Variable`を使う。
- with文の中でGradientTapeを開き、数式間の関係性を定義する
- `GradientTape.gradient`で微分を算出する


In [21]:
w_scalar = tfe.Variable(5.0)
w_matrix = tfe.Variable([[5.0, 4.0], [3.0, 2.0]])

with tf.GradientTape() as tape:
    loss_scalar = w_scalar * w_scalar

print("loss_scalarのw_scalarに対する微分")    
print(tape.gradient(loss_scalar, w_scalar))

with tf.GradientTape() as tape:
    loss_mat = w_matrix * w_matrix
    
print("loss_matrixのw_matrixに対する微分")    
print(tape.gradient(loss_mat, w_matrix))    


tf.Tensor(10.0, shape=(), dtype=float32)
tf.Tensor(
[[10.  8.]
 [ 6.  4.]], shape=(2, 2), dtype=float32)


## 単純モデルの学習
- 線形モデルの回帰


In [12]:
NUM_EXAMPLES = 1000
gt_weight = 4
gt_bias = 3
training_inputs = tf.random_normal([NUM_EXAMPLES])
noise = tf.random_normal([NUM_EXAMPLES])
training_outputs = training_inputs * gt_weight + gt_bias + noise

print("Training Input:")
print(training_inputs)

print("Training Output:")
print(training_outputs)


def prediction(input, weight, bias):
    return input * weight + bias


# Loss Function
def loss(weights, biases):
    error = prediction(training_inputs, weights, biases) - training_outputs
    return tf.reduce_mean(tf.square(error))


def grad(weights, biases):
    with tf.GradientTape() as tape:
        loss_value = loss(weights, biases)
    return tape.gradient(loss_value, [weights, biases])


training_steps = 400
learning_rate = 0.01

W = tfe.Variable(5.)
B = tfe.Variable(10.)

print("Initial Loss: {:.3f}".format(loss(W, B)))

for i in range(training_steps):
    dW, dB = grad(W, B)
    W.assign_sub(dW * learning_rate)
    B.assign_sub(dB * learning_rate)
    if i % 20 == 0:
        print("Loss at step {:03d}: {:.3f}".format(i, loss(W, B)))

print("Final Loss: {:.3f}".format(loss(W, B)))
print("W = {} / W_gt = {}".format(W.numpy(), gt_weight))
print("B = {} / B_gt = {}".format(B.numpy(), gt_bias))



Training Input:
tf.Tensor(
[-9.61849570e-01 -1.27568448e+00  2.89738119e-01 -1.46992192e-01
  1.02971800e-01  5.90518117e-01  3.37335110e-01 -5.68508446e-01
  1.76379576e-01 -1.50329721e+00  8.49860609e-01  6.85996294e-01
 -1.30660757e-01 -1.09452355e+00  4.93334025e-01  1.13318539e+00
  4.61994797e-01 -4.21164572e-01  1.23180974e+00 -5.52534282e-01
  2.99733691e-02  7.29215980e-01  1.15911603e+00 -1.06032169e+00
 -1.13267350e+00  4.97621715e-01  6.62816167e-01 -1.64432514e+00
  1.07079351e+00  1.23702340e-01  1.78932194e-02  1.21842968e+00
  4.86584276e-01  4.41275209e-01  3.98531705e-01 -1.10162199e+00
  2.43237114e+00  2.12306571e+00 -2.75565922e-01 -6.79130852e-01
 -1.27854371e+00 -1.45205033e+00 -1.26508152e+00  9.78441656e-01
 -6.03906333e-01 -1.79386348e-01 -3.32948059e-01  7.43982136e-01
  6.21291287e-02  1.23248421e-01  4.03373778e-01  1.04349786e-02
 -1.60724685e-01  9.06968296e-01  9.94794667e-01 -1.60533559e+00
  2.44425260e-03  9.84883532e-02  9.71781835e-03  5.71241498e-0

Initial Loss: 51.224
Loss at step 000: 49.215
Loss at step 020: 22.319
Loss at step 040: 10.439
Loss at step 060: 5.188
Loss at step 080: 2.866
Loss at step 100: 1.838
Loss at step 120: 1.383
Loss at step 140: 1.181


Loss at step 160: 1.092
Loss at step 180: 1.052
Loss at step 200: 1.035
Loss at step 220: 1.027
Loss at step 240: 1.023
Loss at step 260: 1.022
Loss at step 280: 1.021
Loss at step 300: 1.021


Loss at step 320: 1.021
Loss at step 340: 1.021
Loss at step 360: 1.021
Loss at step 380: 1.021
Final Loss: 1.021
W = 3.9736568927764893 / W_gt = 4
B = 3.0265145301818848 / B_gt = 3


## Mnistの学習
#### 事前準備
- 実行前に、https://github.com/tensorflow/models.gitをcloneしpythonpathを通す
- pycharmのpythonpathの設定はproject interpretorの設定から

In [3]:
import official.mnist

In [8]:
from official.mnist import dataset as mnist_dataset
mnist_dataset.train("C:\Work\Data\mnist")

PermissionError: [WinError 32] プロセスはファイルにアクセスできません。別のプロセスが使用中です。: 'C:\\Users\\kimim\\AppData\\Local\\Temp\\tmp2_fh69og.gz'

In [7]:
!cd


C:\Users\kimim\PycharmProjects\colab_ex
